In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## 데이터 정리

In [ ]:
import os
import shutil
from glob import glob

def move_data(label):
    # 원본 데이터 경로
    base_dir = "/content/drive/MyDrive/DMS/data/YawDD dataset"
    output_dir = f"/content/drive/MyDrive/DMS/data/YawDD dataset/YawDD_preprocessed/{label}"

    # 저장할 폴더 생성
    os.makedirs(output_dir, exist_ok=True)

    # Yawning 비디오만 필터링하여 이동
    for root, _, files in os.walk(base_dir):
        for file in files:
            if label in file and file.endswith(".avi"):
                src_path = os.path.join(root, file)
                dst_path = os.path.join(output_dir, file)
                shutil.move(src_path, dst_path)

    print(f"{label} 비디오 {len(os.listdir(output_dir))}개 이동 완료!")


In [ ]:
move_data("Yawning")
move_data("Normal")
move_data("Talking")

Yawning 비디오 114개 이동 완료!
Normal 비디오 105개 이동 완료!
Talking 비디오 113개 이동 완료!


In [ ]:
move_data("Yawning")

Yawning 비디오 114개 이동 완료!


In [ ]:
move_data("Normal")

Normal 비디오 105개 이동 완료!


In [ ]:
move_data("Talking")

Talking 비디오 113개 이동 완료!


In [ ]:
114+105+113

332

원상복구

2. 데이터 라벨링

In [ ]:
import os
import cv2
import numpy as np

NUM_ELEMENT = 3
NUM_VIDEO_CLASS = 4
TAGS = ['Normal', 'Talking', 'Yawning']
DATA_SET_LIST = ['train_lst', 'test_lst']

for data_set in DATA_SET_LIST:
    # 각 데이터셋에서 하품 구간이 정의된 리스트 파일 경로
    split_file_list = ['./' + data_set + '/male_yawn_split.lst',
                       './' + data_set + '/female_yawn_split.lst',
                       './' + data_set + '/dash_female_yawn_split_yawning.lst',
                       './' + data_set + '/dash_male_yawn_split_yawning.lst']
    # 원본 비디오 경로
    dataset_path_list = ['../Mirror/Male_mirror',
                         '../Mirror/Female_mirror',
                         '../Dash/Female',
                         '../YawDD/Dash/Male']
    # 저장할 경로
    save_path_list = ['./' + data_set + '/mirror_male_split_output',
                      './' + data_set + '/mirror_female_split_output',
                      './' + data_set + '/dash_female_split_output',
                      './' + data_set + '/dash_male_split_output']


    for video_index in range(NUM_VIDEO_CLASS):

        split_file = split_file_list[video_index]
        dataset_path = dataset_path_list[video_index]
        save_path = save_path_list[video_index]

        for i in range(len(TAGS)):
            path = os.path.join(save_path, TAGS[i])
            if os.path.exists(path) is False:
                os.makedirs(path)

        print('split file: ', split_file)
        fd = open(split_file)

        for line in fd.readlines():
            line = line.strip('\n')
            line = line.split(' ')
            video_file = os.path.join(dataset_path, line[0])
            num_clips = int((len(line) - 1) / NUM_ELEMENT)
            # clips_info = np.array(map(int, line[1:]))
            clips_info = np.array([int(x) for x in line[1:]])
            clips_info = clips_info.reshape(num_clips, NUM_ELEMENT)

            cap = cv2.VideoCapture(video_file)
            fcnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
            fps = cap.get(cv2.CAP_PROP_FPS)
            fsize = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                    int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
            print(video_file)
            for i in range(num_clips):
                start_slice = clips_info[i, 0]
                fstart = int(start_slice * fps)
                # if -1, continue go to end
                if clips_info[i, 1] > 0:
                    end_slice = clips_info[i, 1]
                    fend = min(int(end_slice * fps), fcnt)
                else:
                    fend = int(fcnt) - 20
                if fstart >= fend:
                    print('=======start > end, continue======')
                    continue
                print('%d --> %d, tag: %s' % (fstart, fend, TAGS[clips_info[i, 2]]))
                assert(fstart > 0 or fend <= fcnt)
                cap.set(cv2.CAP_PROP_POS_FRAMES, fstart)

                [_, file_name] = os.path.split(video_file)
                [video_name, _] = os.path.splitext(file_name)
                clip_file = '%s/%s/%s-clip-%d.avi' % (save_path, TAGS[clips_info[i, 2]], video_name, i)
                fourcc = cv2.VideoWriter_fourcc('X', 'V', 'I', 'D')
                writer = cv2.VideoWriter(clip_file, fourcc, fps=fps, frameSize=fsize)
                for n in range(fend-fstart):
                    success, img = cap.read()
                    writer.write(img)
                    #cv2.imshow('video', img)
                    #cv2.waitKey(40)
                writer.release()
                #cv2.waitKey(-1)